In [0]:
import pandas as pd

In [0]:
df = pd.read_csv("/content/drive/My Drive/CSV files for Deep learning and ML/Fake news/train.csv")

In [0]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [0]:
# Drop all the NaN values

df = df.dropna()

In [0]:
X = df.drop("label", axis = 1)

In [0]:
y = df["label"]

In [0]:
X.shape

(18285, 4)

In [0]:
y.shape

(18285,)

In [0]:
import tensorflow as tf
tf.__version__

'2.2.0'

#### Things necessary for implementation using LSTM:

1. Embedding Layer
2. Pad Sequences: Whenever we are giving an input to the embedding layer, we have to make sure that the size of the input is fixed. Hence we make use of Pad Sequence (Pad can be added at the beginning of a sentence or at the end of a sentence)
3. Sequential
4. One Hot Encoder
5. LSTM layer
6. Dense layer

In [0]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [0]:
# Vocab size = This is means all the sentences will be framed using these 5000 sentences only

vocab_size = 5000

### One Hot Representation For Title Messages

In [0]:
message = X.copy()

In [0]:
message.reset_index(inplace = True)

In [0]:
import nltk
import re
from nltk.corpus import stopwords

In [0]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
## Data Preprocessing

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
  print(i)
  review = re.sub('[^a-zA-Z]', ' ', message["title"][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words("english")]
  review = " ".join(review)
  corpus.append(review)

In [0]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

- The Title Column has been cleaned

In [0]:
## We will apply One Hot Encoding on this clean data
## One hot encoding will provide a unique number to each word in the corpus
onehot_rep = [one_hot(word, vocab_size) for word in corpus]
onehot_rep

[[3767, 4821, 192, 3194, 4337, 4656, 444, 2567, 3301, 1338],
 [287, 2612, 1050, 3173, 1599, 3493, 4201],
 [3915, 2508, 243, 4183],
 [1993, 4691, 298, 1858, 4984, 4118],
 [78, 1599, 4970, 353, 3547, 2835, 1599, 3750, 4803, 4292],
 [2572,
  1151,
  2015,
  1454,
  1666,
  360,
  231,
  4187,
  190,
  20,
  4534,
  1784,
  1471,
  2875,
  4201],
 [1455, 1986, 1280, 952, 1127, 2478, 1584, 1862, 3260, 4875, 1904],
 [3948, 4190, 4983, 2051, 2880, 960, 360, 351, 3260, 4875, 1904],
 [3002, 2425, 4624, 66, 551, 1509, 2567, 691, 360, 4817],
 [3661, 335, 3259, 2156, 4705, 1664, 881, 2596],
 [4193, 518, 4622, 1393, 3509, 2604, 4923, 2363, 3158, 2890, 4768],
 [1858, 2715, 4337, 1509, 360, 2880],
 [322, 1342, 2616, 925, 1434, 995, 1032, 411, 941],
 [2199, 327, 4286, 4831, 3035, 1840, 140, 3260, 4875, 1904],
 [3874, 3327, 4857, 4200, 3278, 3260, 4875, 1904],
 [4425, 3725, 4713, 1236, 3316, 2589, 4753, 4491, 1811, 1060],
 [3299, 163, 2612],
 [3437, 1204, 2872, 445, 360, 4747, 1293, 4201],
 [1918, 2914

- In the above onehot_rep, each sentence has a different length. Hence before giving it to the embedding layer, we have to give make each sentence of fixed size using padding

### Padding

In [0]:
sent_length = 25
embedded_docs = pad_sequences(onehot_rep, padding= "pre", maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ... 2567 3301 1338]
 [   0    0    0 ... 1599 3493 4201]
 [   0    0    0 ... 2508  243 4183]
 ...
 [   0    0    0 ... 3260 4875 1904]
 [   0    0    0 ... 1805 3203 2128]
 [   0    0    0 ... 2231 4007  580]]


### Creating The LSTM Model

In [0]:
embedded_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size, embedded_vector_features, input_length = sent_length))
model.add(LSTM(100)) # LSTM with 100 neurons
model.add(Dense(1, activation = "sigmoid"))
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [0]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [0]:
X_final.shape, y_final.shape

((18285, 25), (18285,))

In [0]:
# Train test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.33, random_state = 0)

## Final Training

In [0]:
model.fit(X_train, y_train, validation_data= (X_test, y_test),epochs = 10, batch_size = 100)

Epoch 1/10
123/123 [==============================] - 7s 54ms/step - loss: 0.1015 - accuracy: 0.9642 - val_loss: 0.2302 - val_accuracy: 0.9165
Epoch 2/10
123/123 [==============================] - 7s 55ms/step - loss: 0.0781 - accuracy: 0.9727 - val_loss: 0.2585 - val_accuracy: 0.9104
Epoch 3/10
123/123 [==============================] - 7s 54ms/step - loss: 0.0550 - accuracy: 0.9826 - val_loss: 0.2748 - val_accuracy: 0.9185
Epoch 4/10
123/123 [==============================] - 7s 55ms/step - loss: 0.0350 - accuracy: 0.9898 - val_loss: 0.2711 - val_accuracy: 0.9163
Epoch 5/10
123/123 [==============================] - 7s 54ms/step - loss: 0.0247 - accuracy: 0.9930 - val_loss: 0.3468 - val_accuracy: 0.9163
Epoch 6/10
123/123 [==============================] - 7s 53ms/step - loss: 0.0181 - accuracy: 0.9946 - val_loss: 0.3215 - val_accuracy: 0.9143
Epoch 7/10
123/123 [==============================] - 7s 54ms/step - loss: 0.0108 - accuracy: 0.9980 - val_loss: 0.4409 - val_accuracy: 0.9143

### Performance Metrics and Accuracy

In [0]:
y_pred = model.predict_classes(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[3126,  263],
       [ 282, 2364]])

In [0]:
from sklearn.metrics import accuracy_score

In [0]:
accuracy_score(y_test, y_pred)

0.9096934548467275

- In the above model, we can see that the training accuracy reached 100%. This means that there is OVERFITTING of Data.
- Hence we make use of DROPOUT layers

### Adding Dropout

In [0]:
from tensorflow.keras.layers import Dropout
embedded_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size, embedded_vector_features, input_length = sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100)) # LSTM with 100 neurons
model.add(Dropout(0.3))
model.add(Dense(1, activation = "sigmoid"))
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 25, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(X_train, y_train, validation_data= (X_test, y_test),epochs = 10, batch_size = 100)

Epoch 1/10
123/123 [==============================] - 7s 58ms/step - loss: 0.4151 - accuracy: 0.7934 - val_loss: 0.2112 - val_accuracy: 0.9125
Epoch 2/10
123/123 [==============================] - 7s 55ms/step - loss: 0.1641 - accuracy: 0.9341 - val_loss: 0.1969 - val_accuracy: 0.9162
Epoch 3/10
123/123 [==============================] - 7s 54ms/step - loss: 0.1191 - accuracy: 0.9557 - val_loss: 0.1979 - val_accuracy: 0.9176
Epoch 4/10
123/123 [==============================] - 7s 54ms/step - loss: 0.0932 - accuracy: 0.9678 - val_loss: 0.2221 - val_accuracy: 0.9104
Epoch 5/10
123/123 [==============================] - 7s 54ms/step - loss: 0.0801 - accuracy: 0.9722 - val_loss: 0.2276 - val_accuracy: 0.9133
Epoch 6/10
123/123 [==============================] - 7s 56ms/step - loss: 0.0641 - accuracy: 0.9793 - val_loss: 0.2527 - val_accuracy: 0.9143
Epoch 7/10
123/123 [==============================] - 7s 55ms/step - loss: 0.0492 - accuracy: 0.9833 - val_loss: 0.2869 - val_accuracy: 0.9132

In [0]:
y_pred = model.predict_classes(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9148301574150787

In [0]:
# Adding Dropout = 0.5
from tensorflow.keras.layers import Dropout
embedded_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size, embedded_vector_features, input_length = sent_length))
model.add(Dropout(0.5))
model.add(LSTM(100)) # LSTM with 100 neurons
model.add(Dropout(0.5))
model.add(Dense(1, activation = "sigmoid"))
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 25, 40)            200000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 25, 40)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(X_train, y_train, validation_data= (X_test, y_test),epochs = 10, batch_size = 100)

Epoch 1/10
123/123 [==============================] - 7s 56ms/step - loss: 0.4168 - accuracy: 0.7959 - val_loss: 0.2238 - val_accuracy: 0.9107
Epoch 2/10
123/123 [==============================] - 7s 54ms/step - loss: 0.1785 - accuracy: 0.9283 - val_loss: 0.1867 - val_accuracy: 0.9220
Epoch 3/10
123/123 [==============================] - 7s 54ms/step - loss: 0.1331 - accuracy: 0.9506 - val_loss: 0.1943 - val_accuracy: 0.9195
Epoch 4/10
123/123 [==============================] - 7s 55ms/step - loss: 0.1092 - accuracy: 0.9611 - val_loss: 0.2342 - val_accuracy: 0.9216
Epoch 5/10
123/123 [==============================] - 7s 54ms/step - loss: 0.0945 - accuracy: 0.9670 - val_loss: 0.2290 - val_accuracy: 0.9203
Epoch 6/10
123/123 [==============================] - 7s 54ms/step - loss: 0.0800 - accuracy: 0.9740 - val_loss: 0.2578 - val_accuracy: 0.9112
Epoch 7/10
123/123 [==============================] - 7s 54ms/step - loss: 0.0709 - accuracy: 0.9759 - val_loss: 0.2947 - val_accuracy: 0.9186

In [0]:
y_pred = model.predict_classes(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9111847555923778

- Without Dropout Layer we got an accuracy score of 90.9%
- With Dropout(0.3) we got an accuracy score of 91.48%
- With Dropout(0.5) we got an accuracy score of 91.18%